In [379]:
import pandas as pd
import numpy as np

In [380]:
my_data = pd.read_csv("train.csv", index_col = 0)
#print(my_data.head())
#print(my_data.tail())
#print(my_data['arch_and_hp'].head())

In [381]:
my_data = my_data.drop(columns = ['batch_size_test', 'batch_size_val', 'criterion', 'optimizer', 'batch_size_train'])
y_train = my_data['train_error']
y_val = my_data['val_error']
my_data = my_data.drop(columns = 'train_error')
my_data = my_data.drop(columns = 'val_error')
#my_data = my_data.to_numpy()

#print(my_data.head())
x = my_data.drop(columns = ['train_loss','val_loss','arch_and_hp','init_params_mu', 'init_params_std', 'init_params_l2'])
#print(x.head())
col_train = [col for col in x.filter(regex='^train',axis=1).columns]
col_val = [col for col in x.filter(regex='^val',axis=1).columns]

In [382]:
import xgboost as xgb

In [351]:
#1. try on training dataset
x = x.drop(['id'],axis=1)
train = x[col_train]
#val = x[col_val]
val = x

#2. split 70%/30%
split_train_train = train[:int(len(train)*0.7)] #/
split_train_val = val[:int(len(train)*0.7)]

split_y_train_train = y_train[:int(len(train)*0.7)] #/
split_y_train_val = y_val[:int(len(train)*0.7)]

split_test_train = train[int(len(train)*0.7):] #/
split_test_val = val[int(len(train)*0.7):]

split_y_test_train = y_train[int(len(train)*0.7):]
split_y_test_val = y_val[int(len(train)*0.7):]

#3. train
## build Dmatrix
xgtrain = xgb.DMatrix(split_train_train.values, split_y_train_train.values)
xgtest = xgb.DMatrix(split_test_train.values)

## specify parameters via map
param = {'max_depth':3, 'eta':0.5, 'objective':'reg:logistic' }
num_round = 25
bst = xgb.train(param, xgtrain, num_round)
## make prediction
preds_train = bst.predict(xgtest)

#4. val
## build Dmatrix
xgtrain = xgb.DMatrix(split_train_val.values, split_y_train_val.values)
xgtest = xgb.DMatrix(split_test_val.values)

## specify parameters via map
param = {'max_depth':3, 'eta':0.5, 'objective':'reg:logistic' }
num_round = 25
bst = xgb.train(param, xgtrain, num_round)
## make prediction
preds_val = bst.predict(xgtest)

In [352]:
R_square = 1 - (sum((preds_train - split_y_test_train)**2)/sum((preds_train - preds_train.mean())**2))
print(R_square)
R_square = 1 - (sum((preds_val - split_y_test_val)**2)/sum((preds_val - preds_val.mean())**2))
print(R_square)

0.972720817968241
0.8610043450426232


In [383]:
#submit process
#input: train/val for full train dataset

x = x.drop(['id'],axis=1)
train = x[col_train]
#val = x[col_val]
val = x

submit = pd.read_csv("sample_submission.csv", index_col = 0)
submit.Predicted = res_array

test_data = pd.read_csv("test.csv", index_col = 0)
test_data = test_data.drop(columns = ['batch_size_test', 'batch_size_val', 'criterion', 'optimizer', 'batch_size_train'])
test_data = test_data.drop(columns = ['arch_and_hp','init_params_mu', 'init_params_std', 'init_params_l2'])
test_data = test_data.drop(['id'],axis=1)
test_train = test_data[col_train]
#test_val = test_data[col_val]
test_val = test_data

xgtrain = xgb.DMatrix(train.values, y_train.values)
xgtest = xgb.DMatrix(test_train.values)

# specify parameters via map
param = {'max_depth':3, 'eta':0.5, 'objective':'reg:logistic' }
num_round = 25
bst = xgb.train(param, xgtrain, num_round)
# make prediction
preds_train = bst.predict(xgtest)

xgtrain = xgb.DMatrix(val.values, y_val.values)
xgtest = xgb.DMatrix(test_val.values)
# specify parameters via map
param = {'max_depth':3, 'eta':0.5, 'objective':'reg:logistic' }
num_round = 25
bst = xgb.train(param, xgtrain, num_round)
# make prediction
preds_val = bst.predict(xgtest)

#print(preds_train, preds_val)

res_array = np.zeros(preds_train.shape[0]*2)
i = 0
j = 0
while True:
    
    if i == preds_train.shape[0]:
        break
    
    res_array[j] = preds_val[i]
    j += 1
    res_array[j] = preds_train[i]
    j += 1
    
    i += 1 
    
submit.to_csv("submission.csv")